# Preparation

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from pyspark.sql import SparkSession
from pyspark.sql import Row
# from pyspark.sql.types import StringType, StructType, StructField
from pyspark.sql.types import *
# import pyspark.sql.functions as F
from pyspark.sql.functions import *

import warnings
warnings.filterwarnings('ignore')

In [2]:
spark = SparkSession.builder.appName("proj2_q1").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/01 15:04:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/06/01 15:04:14 WARN Utils: Service 'SparkUI' could not bind on port 64050. Attempting port 64051.


# Q1

In [5]:
original_training = spark.read.format("parquet").load("data/squad_v2/train-00000-of-00001.parquet")
original_training.show(5)

+--------------------+-------+--------------------+--------------------+--------------------+
|                  id|  title|             context|            question|             answers|
+--------------------+-------+--------------------+--------------------+--------------------+
|56be85543aeaaa140...|Beyoncé|Beyoncé Giselle K...|When did Beyonce ...|{[in the late 199...|
|56be85543aeaaa140...|Beyoncé|Beyoncé Giselle K...|What areas did Be...|{[singing and dan...|
|56be85543aeaaa140...|Beyoncé|Beyoncé Giselle K...|When did Beyonce ...|     {[2003], [526]}|
|56bf6b0f3aeaaa140...|Beyoncé|Beyoncé Giselle K...|In what city and ...|{[Houston, Texas]...|
|56bf6b0f3aeaaa140...|Beyoncé|Beyoncé Giselle K...|In which decade d...|{[late 1990s], [2...|
+--------------------+-------+--------------------+--------------------+--------------------+
only showing top 5 rows

